In [204]:
import os 
import pandas as pd
import numpy as np

path = os.getcwd()

from sklearn.model_selection import train_test_split

from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, AdaBoostRegressor, BaggingRegressor
from lightgbm import LGBMRegressor
from xgboost import XGBRegressor
from catboost import CatBoostRegressor

from sklearn.model_selection import StratifiedKFold, KFold
from sklearn.preprocessing import StandardScaler, LabelEncoder
le = LabelEncoder()

from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_squared_log_error


In [205]:
path = 'C:\\Users\\sunil\\Projects\\Dockship\\segmind_grand_ai_challenge_2021-dataset'

In [206]:
train = pd.read_csv(path + "\\dataset\\TRAIN.csv")
test = pd.read_csv(path + "\\dataset\\TEST.csv")
ss = pd.read_csv(path + "\\dataset\\sample_submission.csv")
shop = pd.read_csv(path + "\\dataset\\store.csv")

C:\Users\sunil\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (7) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [207]:
test.tail()

,Store,DayOfWeek,Date,Open,Promo,StateHoliday,SchoolHoliday
305158,412,2,2013-05-28,1,1,0,1
305159,936,6,2013-03-02,1,0,0,0
305160,21,1,2013-09-16,1,0,0,0
305161,1077,6,2013-06-22,1,0,0,0
305162,321,6,2013-09-07,1,0,0,0


In [155]:
train['Date'] = pd.to_datetime(train['Date'])
test['Date'] = pd.to_datetime(test['Date'])

In [156]:
train['year'] = train['Date'].dt.year
train['month'] = train['Date'].dt.month
train['day'] = train['Date'].dt.day
train['dayofweek'] = train['Date'].dt.dayofweek
train['week'] = train['Date'].dt.week

test['year'] = test['Date'].dt.year
test['month'] = test['Date'].dt.month
test['day'] = test['Date'].dt.day
test['dayofweek'] = test['Date'].dt.dayofweek
test['week'] = test['Date'].dt.week

train['StateHoliday'].replace({0 : 0, '0':0, 'a':1, 'b':2, 'c':3}, inplace = True)
test['StateHoliday'].replace({0 : 0, '0':0, 'a':1, 'b':2, 'c':3}, inplace = True)

In [157]:
store_type = dict(zip(shop['Store'], shop['StoreType']))
store_ass = dict(zip(shop['Store'], shop['Assortment']))
store_com = dict(zip(shop['Store'], shop['CompetitionDistance']))
store_month = dict(zip(shop['Store'], shop['CompetitionOpenSinceMonth']))
store_year = dict(zip(shop['Store'], shop['CompetitionOpenSinceYear']))
store_p2 = dict(zip(shop['Store'], shop['Promo2']))
store_pweek = dict(zip(shop['Store'], shop['Promo2SinceWeek']))
store_pyear = dict(zip(shop['Store'], shop['Promo2SinceYear']))
store_pi = dict(zip(shop['Store'], shop['PromoInterval']))

In [158]:
train['Store_Type'] = train['Store'].map(store_type)
train['Store_Assortment'] = train['Store'].map(store_ass)
train['Store_distance'] = train['Store'].map(store_com)
train['Store_month'] = train['Store'].map(store_month)
train['Store_year'] = train['Store'].map(store_year)
train['Store_p2'] = train['Store'].map(store_p2)
train['Store_pweek'] = train['Store'].map(store_pweek)
train['Store_pyear'] = train['Store'].map(store_pyear)
train['Store_pi'] = train['Store'].map(store_pi)


test['Store_Type'] = test['Store'].map(store_type)
test['Store_Assortment'] = test['Store'].map(store_ass)
test['Store_distance'] = test['Store'].map(store_com)
test['Store_month'] = test['Store'].map(store_month)
test['Store_year'] = test['Store'].map(store_year)
test['Store_p2'] = test['Store'].map(store_p2)
test['Store_pweek'] = test['Store'].map(store_pweek)
test['Store_pyear'] = test['Store'].map(store_pyear)
test['Store_pi'] = test['Store'].map(store_pi)


train['Store_Assortment'].unique()
train['Store_Assortment'].replace({'a':1, 'b':2, 'c':3}, inplace = True)
test['Store_Assortment'].replace({'a':1, 'b':2, 'c':3}, inplace = True)


a = [0, 'Jan,Apr,Jul,Oct', 'Feb,May,Aug,Nov', 'Mar,Jun,Sept,Dec']
b = [0, 1, 2, 3]

train['Store_pi'].replace(dict(zip(a, b)), inplace = True)
test['Store_pi'].replace(dict(zip(a, b)), inplace = True)

train['Store_Type'].replace({'a':1, 'b':2, 'c':3, 'd':4}, inplace = True)
test['Store_Type'].replace({'a':1, 'b':2, 'c':3, 'd':4}, inplace = True)

In [159]:
train.fillna(0, inplace = True)
test.fillna(0, inplace = True)

In [160]:
target = 'Sales'
date = 'Date'
customers = 'Customers'

features = [col for col in train.columns if col not in [target, date, customers]]

In [161]:
trn, val = train_test_split(train, test_size = 0.2, random_state = 1999)

##### Input for model
X_trn, X_val = trn[features], val[features]

##### Target column
y_trn, y_val = trn[target], val[target]

##### Features for test data that we will be predicting
X_test = test[features]

In [162]:
%%time

lr = LinearRegression()

lr.fit(X_trn, y_trn)

preds = lr.predict(X_val)
preds = np.abs(preds)

error = np.sqrt(mean_squared_error(y_val, preds))

print(f'mean_squared_log_error is : {error}')
#2641

mean_squared_log_error is : 2585.1198192160346
Wall time: 732 ms


In [163]:
%%time

lr = LGBMRegressor(random_state=1, n_estimators=1000)

lr.fit(X_trn, y_trn)

preds = lr.predict(X_val)
preds = np.abs(preds)

error = np.sqrt(mean_squared_error(y_val, preds))

print(f'mean_squared_log_error is : {error}')
#2187

mean_squared_log_error is : 815.575196396159
Wall time: 22.1 s


In [164]:
%%time

lr = XGBRegressor(random_state=1)

lr.fit(X_trn, y_trn)

preds = lr.predict(X_val)
preds = np.abs(preds)

error = np.sqrt(mean_squared_error(y_val, preds))

print(f'mean_squared_log_error is : {error}')
#1440

mean_squared_log_error is : 1132.7221793430826
Wall time: 22.2 s


In [70]:
%%time

lr = RandomForestRegressor(random_state=1)

lr.fit(X_trn, y_trn)

preds = lr.predict(X_val)
preds = np.abs(preds)

error = np.sqrt(mean_squared_error(y_val, preds))

print(f'mean_squared_log_error is : {error}')

mean_squared_log_error is : 1616.327924760229
Wall time: 7min 30s


In [172]:
%%time

lr = CatBoostRegressor(random_state=1)

lr.fit(X_trn, y_trn)

preds = lr.predict(X_val)
preds = np.abs(preds)

error = np.sqrt(mean_squared_error(y_val, preds))

print(f'mean_squared_log_error is : {error}')

Learning rate set to 0.120753
0:	learn: 3649.8729198	total: 86.6ms	remaining: 1m 26s
1:	learn: 3425.1009595	total: 143ms	remaining: 1m 11s
2:	learn: 3240.4944692	total: 207ms	remaining: 1m 8s
3:	learn: 3090.0579405	total: 268ms	remaining: 1m 6s
4:	learn: 2966.4410426	total: 339ms	remaining: 1m 7s
5:	learn: 2867.5702978	total: 434ms	remaining: 1m 11s
6:	learn: 2782.4679788	total: 490ms	remaining: 1m 9s
7:	learn: 2716.4834243	total: 550ms	remaining: 1m 8s
8:	learn: 2659.9132763	total: 680ms	remaining: 1m 14s
9:	learn: 2615.5224044	total: 746ms	remaining: 1m 13s
10:	learn: 2578.3978173	total: 807ms	remaining: 1m 12s
11:	learn: 2546.5572577	total: 865ms	remaining: 1m 11s
12:	learn: 2522.3536442	total: 926ms	remaining: 1m 10s
13:	learn: 2498.7160928	total: 993ms	remaining: 1m 9s
14:	learn: 2477.8748146	total: 1.05s	remaining: 1m 9s
15:	learn: 2458.9999929	total: 1.12s	remaining: 1m 8s
16:	learn: 2446.5814899	total: 1.18s	remaining: 1m 8s
17:	learn: 2435.5866652	total: 1.23s	remaining: 1m 7s

151:	learn: 1799.5555804	total: 12.8s	remaining: 1m 11s
152:	learn: 1796.8084405	total: 12.9s	remaining: 1m 11s
153:	learn: 1793.6185739	total: 13s	remaining: 1m 11s
154:	learn: 1791.4200305	total: 13.1s	remaining: 1m 11s
155:	learn: 1789.5309841	total: 13.2s	remaining: 1m 11s
156:	learn: 1786.8656778	total: 13.3s	remaining: 1m 11s
157:	learn: 1784.6508484	total: 13.4s	remaining: 1m 11s
158:	learn: 1782.7771486	total: 13.5s	remaining: 1m 11s
159:	learn: 1780.6998507	total: 13.6s	remaining: 1m 11s
160:	learn: 1777.0766258	total: 13.7s	remaining: 1m 11s
161:	learn: 1775.3653553	total: 13.8s	remaining: 1m 11s
162:	learn: 1772.6050993	total: 13.8s	remaining: 1m 10s
163:	learn: 1769.9153067	total: 13.9s	remaining: 1m 10s
164:	learn: 1768.4438670	total: 14s	remaining: 1m 10s
165:	learn: 1767.4008201	total: 14s	remaining: 1m 10s
166:	learn: 1766.7240695	total: 14.1s	remaining: 1m 10s
167:	learn: 1763.4767157	total: 14.2s	remaining: 1m 10s
168:	learn: 1760.7995251	total: 14.2s	remaining: 1m 10

301:	learn: 1512.0139291	total: 24.8s	remaining: 57.4s
302:	learn: 1509.8791354	total: 24.9s	remaining: 57.3s
303:	learn: 1507.8696570	total: 25s	remaining: 57.3s
304:	learn: 1506.8133438	total: 25.1s	remaining: 57.2s
305:	learn: 1505.3474112	total: 25.2s	remaining: 57.2s
306:	learn: 1503.6310262	total: 25.3s	remaining: 57.1s
307:	learn: 1500.5466306	total: 25.4s	remaining: 57.1s
308:	learn: 1500.0836395	total: 25.5s	remaining: 57s
309:	learn: 1498.6516691	total: 25.6s	remaining: 56.9s
310:	learn: 1497.0870803	total: 25.6s	remaining: 56.8s
311:	learn: 1495.4988464	total: 25.7s	remaining: 56.7s
312:	learn: 1493.5768558	total: 25.8s	remaining: 56.6s
313:	learn: 1492.6258780	total: 25.9s	remaining: 56.5s
314:	learn: 1491.5619003	total: 25.9s	remaining: 56.4s
315:	learn: 1491.0566865	total: 26s	remaining: 56.3s
316:	learn: 1489.6054907	total: 26.1s	remaining: 56.3s
317:	learn: 1486.9293661	total: 26.2s	remaining: 56.2s
318:	learn: 1485.3906786	total: 26.3s	remaining: 56.1s
319:	learn: 1484

451:	learn: 1329.3992658	total: 36.6s	remaining: 44.4s
452:	learn: 1328.5869108	total: 36.7s	remaining: 44.3s
453:	learn: 1327.7534530	total: 36.8s	remaining: 44.2s
454:	learn: 1326.5615267	total: 36.9s	remaining: 44.1s
455:	learn: 1325.7080400	total: 37s	remaining: 44.1s
456:	learn: 1324.0435346	total: 37s	remaining: 44s
457:	learn: 1323.0861464	total: 37.1s	remaining: 43.9s
458:	learn: 1322.1491034	total: 37.2s	remaining: 43.8s
459:	learn: 1321.1933593	total: 37.3s	remaining: 43.7s
460:	learn: 1319.9484816	total: 37.3s	remaining: 43.6s
461:	learn: 1319.1519592	total: 37.4s	remaining: 43.6s
462:	learn: 1318.5510466	total: 37.5s	remaining: 43.5s
463:	learn: 1317.8445810	total: 37.5s	remaining: 43.4s
464:	learn: 1316.4377981	total: 37.6s	remaining: 43.3s
465:	learn: 1315.3773769	total: 37.7s	remaining: 43.2s
466:	learn: 1313.3474331	total: 37.8s	remaining: 43.1s
467:	learn: 1311.7277493	total: 37.9s	remaining: 43s
468:	learn: 1310.9356417	total: 37.9s	remaining: 42.9s
469:	learn: 1310.0

603:	learn: 1208.9521982	total: 48.2s	remaining: 31.6s
604:	learn: 1208.0438158	total: 48.3s	remaining: 31.5s
605:	learn: 1207.5508062	total: 48.3s	remaining: 31.4s
606:	learn: 1206.9456845	total: 48.4s	remaining: 31.3s
607:	learn: 1206.6648660	total: 48.5s	remaining: 31.3s
608:	learn: 1205.7219551	total: 48.6s	remaining: 31.2s
609:	learn: 1204.9855788	total: 48.6s	remaining: 31.1s
610:	learn: 1204.5297513	total: 48.7s	remaining: 31s
611:	learn: 1203.8029458	total: 48.8s	remaining: 30.9s
612:	learn: 1203.4544910	total: 48.9s	remaining: 30.9s
613:	learn: 1202.7876385	total: 49s	remaining: 30.8s
614:	learn: 1201.6852290	total: 49.1s	remaining: 30.7s
615:	learn: 1200.9966270	total: 49.1s	remaining: 30.6s
616:	learn: 1200.3088057	total: 49.2s	remaining: 30.5s
617:	learn: 1199.5160517	total: 49.3s	remaining: 30.5s
618:	learn: 1198.5049605	total: 49.4s	remaining: 30.4s
619:	learn: 1197.9058629	total: 49.5s	remaining: 30.3s
620:	learn: 1197.4032644	total: 49.5s	remaining: 30.2s
621:	learn: 11

753:	learn: 1125.0735571	total: 59.6s	remaining: 19.4s
754:	learn: 1124.3641902	total: 59.7s	remaining: 19.4s
755:	learn: 1123.8593464	total: 59.7s	remaining: 19.3s
756:	learn: 1123.3065032	total: 59.8s	remaining: 19.2s
757:	learn: 1122.4744815	total: 59.9s	remaining: 19.1s
758:	learn: 1121.9427688	total: 60s	remaining: 19s
759:	learn: 1121.3149071	total: 1m	remaining: 19s
760:	learn: 1120.8632760	total: 1m	remaining: 18.9s
761:	learn: 1120.5590181	total: 1m	remaining: 18.8s
762:	learn: 1120.2632263	total: 1m	remaining: 18.7s
763:	learn: 1120.0345922	total: 1m	remaining: 18.6s
764:	learn: 1119.2182865	total: 1m	remaining: 18.5s
765:	learn: 1118.7282924	total: 1m	remaining: 18.5s
766:	learn: 1118.0644286	total: 1m	remaining: 18.4s
767:	learn: 1117.8388923	total: 1m	remaining: 18.3s
768:	learn: 1117.4968027	total: 1m	remaining: 18.2s
769:	learn: 1117.0401351	total: 1m	remaining: 18.1s
770:	learn: 1116.6103868	total: 1m	remaining: 18.1s
771:	learn: 1116.1062848	total: 1m	remaining: 18s
77

905:	learn: 1062.9506413	total: 1m 10s	remaining: 7.28s
906:	learn: 1062.7481007	total: 1m 10s	remaining: 7.2s
907:	learn: 1062.4814260	total: 1m 10s	remaining: 7.12s
908:	learn: 1062.3732275	total: 1m 10s	remaining: 7.04s
909:	learn: 1061.8196966	total: 1m 10s	remaining: 6.96s
910:	learn: 1061.7106902	total: 1m 10s	remaining: 6.88s
911:	learn: 1061.4883391	total: 1m 10s	remaining: 6.8s
912:	learn: 1061.3153827	total: 1m 10s	remaining: 6.72s
913:	learn: 1060.9852526	total: 1m 10s	remaining: 6.65s
914:	learn: 1060.7806632	total: 1m 10s	remaining: 6.57s
915:	learn: 1060.5704224	total: 1m 10s	remaining: 6.49s
916:	learn: 1060.1764211	total: 1m 10s	remaining: 6.41s
917:	learn: 1059.6403809	total: 1m 10s	remaining: 6.33s
918:	learn: 1059.3930652	total: 1m 10s	remaining: 6.25s
919:	learn: 1059.1389778	total: 1m 11s	remaining: 6.17s
920:	learn: 1058.6925759	total: 1m 11s	remaining: 6.1s
921:	learn: 1058.0002264	total: 1m 11s	remaining: 6.02s
922:	learn: 1057.4740064	total: 1m 11s	remaining: 5

In [72]:
%%time

lr = BaggingRegressor(base_estimator = XGBRegressor(random_state=1), random_state=1, n_estimators = 20, n_jobs = -1)

lr.fit(X_trn, y_trn)

preds = lr.predict(X_val)
preds = np.abs(preds)

error = np.sqrt(mean_squared_error(y_val, preds))

print(f'mean_squared_log_error is : {error}')

mean_squared_log_error is : 1394.411515846407
Wall time: 5min 1s


In [75]:
%%time

lr = BaggingRegressor(base_estimator = DecisionTreeRegressor(random_state=1), random_state=1, n_estimators = 30, n_jobs = -1)

lr.fit(X_trn, y_trn)

preds = lr.predict(X_val)
preds = np.abs(preds)

error = np.sqrt(mean_squared_error(y_val, preds))

print(f'mean_squared_log_error is : {error}')

mean_squared_log_error is : 1648.049807520033
Wall time: 45.9 s


In [192]:
%%time

lr = XGBRegressor(random_state=1)

lr.fit(train[features], train[target])

preds = lr.predict(X_test)
preds = np.abs(preds)

Wall time: 14.1 s


In [193]:
test['Predictions'] = preds

In [195]:
test.loc[ test['Open'] == 0, 'Predictions'] = 0

In [196]:
preds = test['Predictions']

In [197]:
index = [i for i in range(test.shape[0])]

d = list(zip(index, preds))

ss = pd.DataFrame(d, columns = ['index', 'Sales'])

In [198]:
ss.to_csv(path + "\\baseline_xsb.csv", index = False)

In [199]:
ss['Sales'].min()

0.0